# Analyzing unstructured customer reviews with Snowflake Cortex
Understanding customer feedback is critical for businesses, but analyzing large volumes of unstructured text can be challenging. In this notebook, you'll use Snowflake Cortex to perform **sentiment** analysis

By the end, you'll see how Snowflake can help extract insights from unstructured data quickly and effectively.

### Context
*Tasty Bytes* is a global food truck network operating in 15 countries with a fleet of 450 trucks. They collect customer reviews to gain insights into their food truck operations, with feedback coming from multiple sources.

In this notebook, we will analyze these collated customer reviews using Snowflake Cortex to understand what our international customers are saying.

##

## Import sample data

In this next SQL query, we will populate sample data that is used in this and other templates.

In [ ]:
USE ROLE SNOWFLAKE_LEARNING_ROLE;

-- use the existing database, schema and warehouse
USE DATABASE SNOWFLAKE_LEARNING_DB;

SET schema_name = CONCAT(current_user(), '_ANALYZE_SENTIMENT_IN_UNSTRUCTURED_DATA');
USE SCHEMA IDENTIFIER($schema_name);

  /*--
  • file format and stage creation
  --*/

  CREATE OR REPLACE FILE FORMAT csv_ff 
    TYPE = 'csv';

  CREATE OR REPLACE STAGE s3load
    COMMENT = 'Quickstarts S3 Stage Connection'
    URL = 's3://sfquickstarts/tastybytes-voc/'
    FILE_FORMAT = csv_ff;


  /*--
  • raw zone table build 
  --*/

  -- truck_reviews table
  CREATE OR REPLACE TABLE truck_reviews
  (
      order_id NUMBER(38,0),
      language VARCHAR(16777216),
      source VARCHAR(16777216),
      review VARCHAR(16777216),
      review_id NUMBER(18,0)
  );
  
  /*--
  • raw zone table load 
  --*/
  
  -- truck_reviews table load
  COPY INTO truck_reviews
  FROM @s3load/raw_support/truck_reviews/;

-- setup completion note
SELECT 'Setup is complete' AS note;

## Import python packages

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel. 

Now that the necessary packages are installed, we will import the installed packages into the notebook.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()

## Overview of Reviews Table

In [ ]:
SELECT ORDER_ID, LANGUAGE, REVIEW FROM TRUCK_REVIEWS LIMIT 15;

## Sentiment Analysis
Customer reviews can provide valuable feedback, but understanding their overall sentiment is crucial for identifying trends. With Snowflake Cortex, we can automatically determine whether a review expresses a **positive, neutral, or negative** sentiment. 

Cortex Sentiment will return a number ranging from -1 to 1 indicating the level of negative (-1) or positive (1) sentiment in the text. Values around 0 indicate neutral sentiment, while numbers further from 0 indicate strength of sentiment. 

We can perform sentiment analysis using Cortex both in Python on a Dataframe, or each cell in a column of a Snowflake table with SQL.

### Analyze sentiment on a Python dataframe

Cortex Sentiment can be used in Python or SQL. The following Python code will:
1. Extract review text and apply **Cortex's `cortex.sentiment()` function** to determine the sentiment.
2. Display the first few results to observe the breakdown of customer sentiment.

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = session.table('truck_reviews')

reviews_df = reviews_df.withColumn('SENTIMENT', cortex.sentiment(F.col('REVIEW')))

reviews_df.select(["REVIEW","SENTIMENT"]).show(15, max_width = 100)

### Analyze sentiment with a SQL query

In [ ]:
SELECT 
  REVIEW_ID,
  REVIEW,
  SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) AS SENTIMENT
FROM TRUCK_REVIEWS
LIMIT 15;

# Conclusion
In this template, we've demonstrated how to analyze unstructured customer reviews using Snowflake Cortex. 

By following these steps, you now have a replicable process for understanding whether customer reviews express positive, neutral, or negative sentiments.

To learn more about Cortex Sentiment or Cortex LLM Functions, check out the [Snowflake Cortex documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) and the [Cortex Sentiment documentation](https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex).
